In [8]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, classification_report

from sklearn.linear_model import LogisticRegression
import xgboost as xgb

import shap
import matplotlib.pyplot as plt

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)


In [9]:
df = pd.read_csv("/content/Sepsis Prediction.csv")

# Basic structural check
df.info()
df.head()

# Outcome distribution
df["SepsisLabel"].value_counts()
df["SepsisLabel"].value_counts(normalize=True)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1552210 entries, 0 to 1552209
Data columns (total 44 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Unnamed: 0        1552210 non-null  int64  
 1   Hour              1552210 non-null  int64  
 2   HR                1398811 non-null  float64
 3   O2Sat             1349474 non-null  float64
 4   Temp              525226 non-null   float64
 5   SBP               1325945 non-null  float64
 6   MAP               1358940 non-null  float64
 7   DBP               1065656 non-null  float64
 8   Resp              1313875 non-null  float64
 9   EtCO2             57636 non-null    float64
 10  BaseExcess        84145 non-null    float64
 11  HCO3              65028 non-null    float64
 12  FiO2              129365 non-null   float64
 13  pH                107573 non-null   float64
 14  PaCO2             86301 non-null    float64
 15  SaO2              53561 non-null    float64
 16  

,proportion
SepsisLabel,
0,0.982015
1,0.017985


In [10]:
# Drop non-informative index column and fully-missing feature
df = df.drop(columns=["Unnamed: 0", "EtCO2"])

y = df["SepsisLabel"]
patient_ids = df["Patient_ID"]

X = df.drop(columns=["SepsisLabel", "Patient_ID"])



In [11]:
df_model = X.copy()
df_model["SepsisLabel"] = y
df_model["Patient_ID"] = patient_ids

agg_stats = ["mean", "max", "min"]

patient_features = (
    df_model
    .groupby("Patient_ID")
    .agg({col: agg_stats for col in X.columns})
)

patient_features.columns = [
    f"{col}_{stat}" for col, stat in patient_features.columns
]

patient_labels = (
    df_model
    .groupby("Patient_ID")["SepsisLabel"]
    .max()
)

X_patient = patient_features
y_patient = patient_labels

X_patient.shape
y_patient.value_counts(normalize=True)


,proportion
SepsisLabel,
0,0.927311
1,0.072689


In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X_patient,
    y_patient,
    test_size=0.25,
    stratify=y_patient,
    random_state=RANDOM_SEED
)


In [13]:
# Explicit missingness indicators
X_train_missing = X_train.isna().astype(int)
X_test_missing = X_test.isna().astype(int)

X_train_missing.columns = [c + "_missing" for c in X_train.columns]
X_test_missing.columns = [c + "_missing" for c in X_test.columns]

# Median imputation
imputer = SimpleImputer(strategy="median")

X_train_imp = pd.DataFrame(
    imputer.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)

X_test_imp = pd.DataFrame(
    imputer.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

# Final feature matrices
X_train_final = pd.concat([X_train_imp, X_train_missing], axis=1)
X_test_final = pd.concat([X_test_imp, X_test_missing], axis=1)

X_train_final.shape, X_test_final.shape
X_train_final.isna().any().any()


np.False_

In [14]:
log_reg = LogisticRegression(
    max_iter=2000,
    class_weight="balanced",
    n_jobs=-1
)

log_reg.fit(X_train_final, y_train)

y_test_prob_lr = log_reg.predict_proba(X_test_final)[:, 1]

roc_auc_score(y_test, y_test_prob_lr)
classification_report(
    y_test,
    (y_test_prob_lr >= 0.5).astype(int)
)


'              precision    recall  f1-score   support\n\n           0       0.98      0.78      0.87      9351\n           1       0.22      0.77      0.34       733\n\n    accuracy                           0.78     10084\n   macro avg       0.60      0.77      0.60     10084\nweighted avg       0.92      0.78      0.83     10084\n'

In [15]:
# Ensure identical feature ordering
X_test_final = X_test_final[X_train_final.columns]

xgb_model = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum(),
    random_state=RANDOM_SEED,
    n_jobs=-1,
    eval_metric="logloss"
)

xgb_model.fit(X_train_final, y_train)

y_test_prob = xgb_model.predict_proba(X_test_final)[:, 1]

roc_auc_score(y_test, y_test_prob)



np.float64(0.9398538111134309)

In [16]:
# Report performance at a clinically interpretable operating point
threshold = 0.30

y_pred = (y_test_prob >= threshold).astype(int)

classification_report(y_test, y_pred)



'              precision    recall  f1-score   support\n\n           0       0.99      0.88      0.93      9351\n           1       0.36      0.84      0.50       733\n\n    accuracy                           0.88     10084\n   macro avg       0.67      0.86      0.72     10084\nweighted avg       0.94      0.88      0.90     10084\n'

In [17]:
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_test_final)

# Global importance (bar)
shap.summary_plot(
    shap_values,
    X_test_final,
    plot_type="bar",
    show=False
)

plt.savefig(
    "figures/shap_global_bar.png",
    dpi=300,
    bbox_inches="tight"
)
plt.close()


/tmp/ipython-input-899955069.py:5: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(
